In [2]:
import mlflow
mlflow.set_experiment('exp 5 - hp')
mlflow.set_tracking_uri('http://18.116.112.58:5000')

2024/10/30 16:52:53 INFO mlflow.tracking.fluent: Experiment with name 'exp 5 - hp' does not exist. Creating a new experiment.


In [3]:
!pip install optuna

     |████████████████████████████████| 362 kB 7.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/harsimranjitsingh/Desktop/youtube_comment_analysis/myenv/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

/Users/harsimranjitsingh/Desktop/youtube_comment_analysis/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('/Users/harsimranjitsingh/Desktop/youtube_comment_analysis/notebooks/dataset.csv')
df.dropna(inplace=True)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()


[I 2024-10-30 16:53:50,334] A new study created in memory with name: no-name-25a83b60-4b7c-44e8-a9c0-248d2d062123
[I 2024-10-30 16:59:03,224] Trial 0 finished with value: 0.8229761149862609 and parameters: {'C': 1.2701535042522643, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8229761149862609.
[I 2024-10-30 17:04:42,885] Trial 1 finished with value: 0.40720777848235046 and parameters: {'C': 0.005349264210720577, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8229761149862609.
[I 2024-10-30 17:06:53,085] Trial 2 finished with value: 0.7454026632847178 and parameters: {'C': 0.16068901467998134, 'kernel': 'linear'}. Best is trial 0 with value: 0.8229761149862609.
[I 2024-10-30 17:10:00,702] Trial 3 finished with value: 0.44831959416613826 and parameters: {'C': 0.00039856637860507713, 'kernel': 'linear'}. Best is trial 0 with value: 0.8229761149862609.
[I 2024-10-30 17:13:09,804] Trial 4 finished with value: 0.3628196998520397 and parameters: {'C': 0.0012309866962127672, 'kernel': 'pol